# Error Analysis on 8760 profiles (22 profiles)

In [7]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs\error_analysis'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

##UNCOMMENT WHICH PROFILE BEING ANALYZED 
x = 'load'
x2 = 'Load'

#x = 'solar'
#x2 = 'Solar_Gen'

#x = 'wind'
#x2 = 'Wind_Gen'

profiles = ['norm','timeofday','bind_1hr','seq_2hr','seq_4hr','seq_8hr','seq_12hr','seq_24hr','seq_48hr',
           'seq_120hr','bind_8760hr','1dt_mon_24hr','1dt_mon_4hr','1dt_sea_24hr','2dt_mon_24hr','2dt_sea_24hr',
            '2dt_bim_4hr','2dt_sea_4hr','2dt_sgp_4hr','3dt_mon_24hr','3dt_sea_24hr','3dt_ann_24hr','3dt_bim_4hr',
            '3dt_sgp_4hr','3dt_sea_4hr']

output files are written out in parent directory: C:\Users\tgoforth\Documents\IPM temporal resolution project\outputs\error_analysis


In [8]:
# create DFs for error analysis
# DF to use for regional error analysis, like RMSE and number of representative hours 
regions = pd.read_csv('../outputs/load_long_format.csv')
regions = regions['Region'].unique()
regions = pd.DataFrame(regions).rename(columns={0:'Region'})
#print(regions)

## Loop and create error files 

In [9]:
rep_hours = regions
reg_RMSE = regions
prof_RMSE = {}

for i in range(len(profiles)):
    segments = pd.read_csv('../outputs/'+x+'/'+x+'_segments_'+profiles[i]+'.csv')
    hours = pd.read_csv('../outputs/'+x+'/'+x+'_8760_'+profiles[i]+'.csv')
    
    # use groupby region to count number of representative hours in each region
    aggregations = {'Avg':'count'}
    stat1 = segments.groupby(['Region'],as_index=False).agg(aggregations)
    stat1 = stat1.rename(columns={'Avg': profiles[i]})
    
    rep_hours = pd.merge(rep_hours, stat1, on='Region', how='left')
    
    # calculate RMSE 
    stat2 = hours[['Region','R_Group','Hour_Tot',x2,'Avg']].copy()
    
    # export differences to CSV
    stat2['Diff'] = stat2[x2] - stat2['Avg'] 
    stat2.to_csv('../outputs/error_analysis/'+x+'_'+profiles[i]+'_diff.csv')
    
    # regional RMSE
    stat3 = stat2.copy()
    stat3['Square'] = stat3['Diff']**2
    stat_reg = stat3.groupby('Region',as_index=False).agg({'Square' : sum})
    stat_reg = stat_reg.rename(columns={'Square':'Square_Sum'})
    stat_reg['Mean'] = stat_reg['Square_Sum'] / 8760 
    stat_reg[profiles[i]] = stat_reg['Mean']**(1/2)
    stat_reg = stat_reg.drop(columns={'Square_Sum','Mean'})

    reg_RMSE = pd.merge(reg_RMSE, stat_reg, on='Region', how='left')
    
    # profile RMSE
    stat4 = stat2.copy()
    stat4 = stat4.rename(columns={'Square':'RMSE'})
    stat4['RMSE'] = stat2['Diff']**2
    stat4 = stat4.agg({'RMSE' : sum})
    stat4 = stat4 / 551880 
    stat4[profiles[i]] = stat4**(1/2)
    stat4[profiles[i]] = stat4[profiles[i]].values

    prof_RMSE.update({profiles[i] : (stat4[profiles[i]])})
    
rep_hours.to_csv('../outputs/error_analysis/rep_hours_summary.csv')
reg_RMSE.to_csv('../outputs/error_analysis/regional_RMSE.csv')
profile_df = pd.DataFrame.from_dict(prof_RMSE,orient='index').rename(columns={0:'RMSE'})
profile_df.to_csv('../outputs/error_analysis/profile_RMSE.csv')